#### ----An example with tf.placeholder:----

In [5]:
import numpy as np
import tensorflow as tf

x = tf.placeholder(tf.float32, shape =(1024,1024))
y = tf.matmul(x,x)

with tf.Session() as sess:
    rand_array = np.random.rand(1024,1024)
    print(sess.run(y, feed_dict={x:rand_array}))  #x tensor is feeded with the rand_array 



[[257.45944 254.60678 258.78598 ... 259.38147 258.58023 256.01477]
 [264.3697  257.96545 253.21733 ... 266.1487  257.69424 260.34152]
 [260.25763 250.60455 257.49536 ... 262.05325 257.29956 257.1919 ]
 ...
 [250.00182 252.39375 254.70703 ... 257.2407  251.31036 260.2009 ]
 [260.4658  255.94916 252.51193 ... 255.64287 256.9048  258.2929 ]
 [262.93042 254.96632 257.22543 ... 265.0838  260.0514  256.20804]]


#### ---Example with tensor.get_shape() and tensor.reshape()--- 

In [22]:
sess = tf.Session()
tensor = tf.convert_to_tensor(np.array([[1001,1002,1003],[3,4,5]]),dtype=tf.float32)

sess.run(tensor)

tensor_shape = tensor.get_shape()
tensor_shape

tensor2 = tf.reshape(tensor, tf.TensorShape([6,1]))

print("------------")
print("shape tensor 1: ",tensor_shape)
print("type: ",type(tensor_shape))
print("-------------")
print("shape tensor 2: ", tensor2.get_shape().as_list())
print("type: ",type(tensor.get_shape().as_list()))
print("Number of column: ",int(tensor.get_shape()[1]))

------------
shape tensor 1:  (2, 3)
type:  <class 'tensorflow.python.framework.tensor_shape.TensorShape'>
-------------
shape tensor 2:  [6, 1]
type:  <class 'list'>
Number of column:  3


### +++ Modularity: +++

#### Function to build a ReLU: 

In [13]:
import tensorflow as tf

def relu(X):
    with tf.name_scope("relu"):
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name ="threshold")
        w_shape = (int(X.get_shape()[1]),1)
        w = tf.Variable(tf.random_normal(w_shape, name = "weights"))
        X = tf.placeholder(tf.float32,shape=(None, n_features), name ="X") # a tensor with shape: (?,n_fatures)
        z = tf.add(tf.matmul(X, w), relu.threshold, name ="z") #shape: (?,1) with ? number of rows for X variable
        return tf.maximum(z, relu.threshold, name ="relu")

#### Adding 5 ReLu togheter: (and creating a graph in Tensorboard)

In [14]:
from datetime import datetime
import numpy as np

n_features = 3
now = datetime.utcnow().strftime("%Y%m%d%H%M%S") #now = time in a specific format (YYYYMMDDHHMMSS)
root_logdir = "tf_logs"
logdir = "{}/run-{}".format(root_logdir, now )

X = tf.placeholder(tf.float32, shape=(None, n_features), name ="X")

relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
file_writer.close()

#### Using Perceptron algo for classify :

In [25]:
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris()
X = iris.data[:,(2,3)] #petal length and petal witdth
y = (iris.target == 0).astype(np.int)
per_clf = Perceptron(random_state=42)
per_clf.fit(X,y)

y_pred = per_clf.predict([[2,0.5]])

C:\Users\nabu-pc\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


## Training a DNN : (Deep Neural Network):

#### We use the MNIST DB. It contains handwritten digits. <br> <br> Training set: 60 000 examples. Test set: 10 000 examples.<br> <br> It's a subset of a larger set named NIST. The digits have been size normalized and centered in a fixed-size image.  <br> <br> We have image of 28 X 28 pixel   with grey levels, not B/W image.

### --- Contruction Phase:---

#### Defining the variables

In [24]:
from datetime import datetime
import numpy as np
import tensorflow as tf

n_inputs = 28*28  #MNIST dataset.  . 
n_hidden1 = 300  #number of hidden neurons for layer 1 and 2 
n_hidden2 = 100
n_outputs = 10

now = datetime.utcnow().strftime("%Y%m%d%H%M%S") #now = time in a specific format (YYYYMMDDHHMMSS)
root_logdir = "tf_logs"
logdir = "{}/run-{}".format(root_logdir, now ) # The path where we save the data: tf_logs/run-YYYYMMDDHHMMSS

X = tf.placeholder(tf.float32, shape=(None, n_inputs),name="X") #--The TRAINING BATCH (X)-- shape: (None, n_inputs)
                                                                # X acts as "input layer". 
                                                                #It will be replaced with one training batch at the time
y = tf.placeholder(tf.int64, shape=(None), name="y") # --y-- shape : (None)

### Neuron layer function:

#### We have different parameters: inputs, number of neurons, the activation function and the name of the layer 

In [2]:
def neuron_layer(X,n_neurons, name, activation = None):
    with tf.name_scope(name):  #name_scope is useful in Tensorboard
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs,n_neurons),stddev = stddev)
        W = tf.Variable(init, name ="kernel") # The kernel layer is a matrix (2d tensor )that holds the WEIGHTS
                                              # It's contains all connection weights between each input and each neuron
                                              # Its shape: (n_inputs,n_neurons)
                                              # it will be initialized randomly, using a truncated normal distribution with stdev
                                              # With this specific stddev the algorithm converge faster
        b = tf.Variable(tf.zeros([n_neurons]), name="bias") #For biases. Initialized to 0. One bias parameter per neuron
        Z = tf.matmul(X,W) + b # Shape : (None, n_neurons)
                               # This vectorized implementation will efficiently compute the weight sums of the inputs plus bias  
                               # for each neuron in the layer, for all instances in the batch in one shot.
        if activation is not None:
            return activation(Z)     # if an activation parameter is provided, such as tf.nn.relu (i.e. max(0,Z)), the function 
        else:                        # returns the activation(Z), else returns Z
            return Z
        

### Creating different layers:

In [25]:
import numpy as np

with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X,n_hidden1,name="hidden1", activation=tf.nn.relu)
    
    hidden2 = neuron_layer(hidden1,n_hidden2, name = "hidden2", activation=tf.nn.relu)
    
    logits = neuron_layer(hidden2, n_outputs, name="outputs") # the output of the neural networks before the activation 
                                                              # function softmax. For optimization reasons softmax will be compute later
    
  # We can also create layer without neuron_layer() function. Tensorflow include method to create fully connected layers
  # where all inputs are connected to all neurons in the layer
    
  # hidden1 = tf.layers.dense(X,n_hidden1, name="hidden1", activation=tf.nn.relu)
  # hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation= tf.nn.relu)
  # logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    

### Calculating the Cost function (Cross Entropy)/ Gradient Optimizer / Accuracy: 

#### We define the Cross Entropy as the Cost function that we use to train the nn. <br> <br> Weak aspect :  this cross function will penalize models that estimate a low probability for the target class.

In [26]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits = logits)
    # It calculates the cross entropy based on the "logits" (the outpute before going through the softmax activation function)
    # It expects labels in the form of integers ranging from 0 to the number of classes minus 1 (0-9)
    #The output is 1D tensor with the cross entropy for each istance.
    loss = tf.reduce_mean(xentropy, name="loss") #it computes the mean cross entropy over all instances

#### As optimizer we use the Gradient Descent in order to minimize the Cost function

In [27]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

#### We calculate the accuracy of our our model

In [28]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y,1) #For each instance it determines if the nn's prediction is correct by checking wheter 
                                          #or not the highest logit corresponds to the target class
                                          #It returns a 1d Tensor full of boolean values
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) 

#### Initializing / save phase

In [29]:
init = tf.global_variables_initializer() #We create a node to initialize all variables
saver = tf.train.Saver() # We will create a Saver to save our trained model parameters to disk

loss_summary = tf.summary.scalar('LOSS', loss) # Creates a node in the graph that will evaluate the MSE value and writes it to a TensoBoard-compatible binary log string called Summary
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph()) # We create a filewriter that you use to write summaries to logfiles in the log directory
                                                                    # logdir = the patph of the logdir diretory. Second parameter = the graph we want to visualize

### ---Execution Phase:---

#### We import data :

In [9]:
from  tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/") # the minst object has different method. Some methods can extract 
                                                             # the training istance (50k instance), a validation set(5k instances)
                                                             # a test set(10k)
n_epochs = 40  # numbers of eboch we want to run
batch_size = 50 

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [30]:
n_batches = mnist.train.num_examples // batch_size


with tf.Session() as sess:
    init.run()  #an initial node that initializes all the variables
    for epoch in range(n_epochs): 
        for iteration in range(mnist.train.num_examples // batch_size):  # // operator returns returns int(a/b)
            X_batch, y_batch = mnist.train.next_batch(batch_size)      # for each epoch the algo uses all batchs in the dataset (the whole training size)
            summary_str = loss_summary.eval(feed_dict={X: X_batch, y: y_batch})
            step = epoch * n_batches + iteration  
            file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict = {X: X_batch, y: y_batch}) 
        acc_train = accuracy.eval(feed_dict = {X: X_batch, y: y_batch})#Here X_bath and y_batch come from last bath in Training Set
        acc_val = accuracy.eval(feed_dict = {X: mnist.validation.images, y: mnist.validation.labels}) #the algo is evaluated on 
                                                                                                      #the full validation set
        # For TensorBoard purpose is better to calculate the accuracy only on acc_train!!!! 
        
        print(epoch, "Train accuracy", acc_train, "Test accuracy", acc_val) 
        
        
    save_path = saver.save(sess, "/tmp/my_model_final2.ckpt")
    file_writer.close()
    
# For running Tensorboard, in the Anaconda Prompt IN /Progetto Forex/Python (our working directory) -> tensorboard --logdir tf_logs

0 Train accuracy 0.92 Test accuracy 0.9124
1 Train accuracy 0.96 Test accuracy 0.9292
2 Train accuracy 0.96 Test accuracy 0.9372
3 Train accuracy 1.0 Test accuracy 0.9448
4 Train accuracy 0.94 Test accuracy 0.95
5 Train accuracy 0.92 Test accuracy 0.956
6 Train accuracy 0.96 Test accuracy 0.9604
7 Train accuracy 0.94 Test accuracy 0.9618
8 Train accuracy 0.94 Test accuracy 0.964
9 Train accuracy 0.92 Test accuracy 0.9638
10 Train accuracy 0.96 Test accuracy 0.9674
11 Train accuracy 1.0 Test accuracy 0.967
12 Train accuracy 0.96 Test accuracy 0.9674
13 Train accuracy 0.98 Test accuracy 0.9698
14 Train accuracy 1.0 Test accuracy 0.9698
15 Train accuracy 0.98 Test accuracy 0.9712
16 Train accuracy 0.96 Test accuracy 0.9718
17 Train accuracy 0.96 Test accuracy 0.973
18 Train accuracy 0.96 Test accuracy 0.973
19 Train accuracy 0.98 Test accuracy 0.974
20 Train accuracy 1.0 Test accuracy 0.9724
21 Train accuracy 1.0 Test accuracy 0.9722
22 Train accuracy 1.0 Test accuracy 0.973
23 Train accu

### ---Using the Neural Network:---

#### Now the nn is trained. we can use it to make predictions. 

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(mnist.test.images)   #We need to scaling the images. Each images has 28*28 pixel -> 784 pixels.Each pixel a range(-1,1)
images_scaled =scaler.transform(mnist.test.images)


with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final2.ckpt") #We load the model parameters from the disk
    Z = logits.eval(feed_dict={X: images_scaled}) #We evaluate the logits node
    y_pred = np.argmax(Z,axis =1) # For predicting a class we pick the class that has the highest logit value (argmax())
    
    # ---IMP!!!---if we want to know all estimates class probabilities we need to apply the the softmax() function to the logits

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


#### Printing the first 30 predictions of Test Set:

In [27]:
print(y_pred[:30])

[7 2 1 0 4 1 4 5 4 7 0 6 9 0 1 5 7 7 3 4 7 6 6 5 4 0 7 4 0 1]
